In [2]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import mne

Defining the triggers and have a look at their timing

In [39]:
epochs = mne.read_epochs("../sample/epochs/asjt-epo.fif.gz", preload=True)
even_ids = epochs.event_id
even_ids.pop("New Segment/", None)
fig, ax = plt.subplots(1, 1, figsize=(10, 4), layout="tight")
mne.viz.plot_events(epochs.events, sfreq=1000, event_id=even_ids, axes=ax)
ax.get_legend().remove()
ax.spines[["right", "top"]].set_visible(False)

Reading /Users/payamsadeghishabestari/TinReg/test/../sample/epochs/asjt-epo.fif.gz ...


KeyboardInterrupt: 

a. train a classifier on the random condition (least predictable)

b. test it on ordered sequences


*This allows you to measure how much the brain's predictive pattern generalizes to more predictable contexts—essentially quantifying prediction strength.

In [ ]:
## splitting the epochs into rand and ord
rnd_ids = [key for key in even_ids if key.endswith("rndm")]
ord_ids = [key for key in even_ids if key.endswith("or")]
epochs_rnd = epochs[rnd_ids]
epochs_ord = epochs[ord_ids]



{'f1_std_or': 1,
 'f2_std_or': 2,
 'f3_std_or': 3,
 'f4_std_or': 4,
 'f1_std_rndm': 5,
 'f2_std_rndm': 6,
 'f3_std_rndm': 7,
 'f4_std_rndm': 8,
 'f1_tin_or': 11,
 'f2_tin_or': 12,
 'f3_tin_or': 13,
 'f4_tin_or': 14,
 'f1_tin_rndm': 15,
 'f2_tin_rndm': 16,
 'f3_tin_rndm': 17,
 'f4_tin_rndm': 18}

In [44]:
epochs_random

<EpochsFIF | 3000 events (all good), -1 – 1 s (baseline -1 – 0 s), ~2.95 GiB, data loaded,
 'f1_std_rndm': 365
 'f2_std_rndm': 377
 'f3_std_rndm': 368
 'f4_std_rndm': 390
 'f1_tin_rndm': 358
 'f2_tin_rndm': 383
 'f3_tin_rndm': 395
 'f4_tin_rndm': 364>

In [ ]:
rnd_ids = [key for key in trigger_dict if key.endswith("rndm")]
epochs_random = epochs[rnd_ids]

In [ ]:
## select only random epochs
random_epochs = epochs['random']  # if you labeled them during epoching
X_train = random_epochs.get_data()  # shape: n_trials x n_channels x n_times
y_train = random_epochs.events[:, 2]  # the class labels

In [ ]:
## set up classifier
from sklearn.linear_model import LogisticRegression
from mne.decoding import GeneralizingEstimator

clf = LogisticRegression(max_iter=1000, solver='liblinear')
# time-generalization (optional)
time_gen = GeneralizingEstimator(clf, scoring='accuracy', n_jobs=1)
time_gen.fit(X_train, y_train)

In [ ]:
## test on predictive
full_epochs = epochs['full']  # 100% predictable
X_test = full_epochs.get_data()
y_test = full_epochs.events[:, 2]

scores = time_gen.score(X_test, y_test)  # decoding accuracy over time

In [ ]:
## extract clf weights
import numpy as np

# weights per channel
weights = np.mean(time_gen.estimators_[0].coef_, axis=0)  # average across classes
# can plot topomap
mne.viz.plot_topomap(weights, epochs.info)

In [ ]:
import mne
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score, StratifiedKFold
from mne.decoding import SlidingEstimator, cross_val_multiscore, GeneralizingEstimator

# --- 1. Define your epochs and events ---
# epochs: MNE Epochs object with your data
# trigger_dict: dictionary mapping frequency to integer event ID
# Example: trigger_dict = {'f500Hz': 1, 'f1000Hz': 2, 'f1500Hz': 3, 'f2000Hz': 4}

# Select only the random sequence epochs for training
random_epochs = epochs['random']  # adjust if you have a metadata column
X_train = random_epochs.get_data()  # shape: (n_epochs, n_channels, n_times)
y_train = random_epochs.events[:, 2]  # event IDs

# --- 2. Time-resolved decoding ---
lda = LinearDiscriminantAnalysis()
time_decod = SlidingEstimator(lda, scoring='accuracy', n_jobs=-1)
scores = cross_val_multiscore(time_decod, X_train, y_train, cv=5)  # 5-fold CV
scores_mean = np.mean(scores, axis=0)  # mean over CV folds

# --- 3. Temporal Generalization ---
gen_estimator = GeneralizingEstimator(lda, scoring='accuracy', n_jobs=-1)
gen_estimator.fit(X_train, y_train)  # fit on post-stimulus
temporal_gen_scores = gen_estimator.score(X_train, y_train)  # time x time matrix

# --- 4. Cross-decoding: train on random, test on ordered ---
ordered_epochs = epochs['ordered']
X_test = ordered_epochs.get_data()
y_test = ordered_epochs.events[:, 2]

lda.fit(X_train, y_train)
cross_decoding_scores = np.array([lda.score(X_test[:, :, t], y_test) for t in range(X_test.shape[2])])

# --- 5. Train post-stimulus, test pre-stimulus (random tones) ---
# Assuming epochs.time gives the time vector
post_idx = np.where(epochs.times >= 0)[0]  # post-stimulus
pre_idx = np.where(epochs.times < 0)[0]   # pre-stimulus

X_post = X_train[:, :, post_idx]
X_pre = X_train[:, :, pre_idx]

lda.fit(X_post, y_train)
pre_stim_scores = np.array([lda.score(X_pre[:, :, t], y_train) for t in range(X_pre.shape[2])])

# --- 6. Optional: average across subjects if you have multiple subjects ---
# Store scores for each subject and compute mean at the group level

print("Time-resolved decoding scores shape:", scores_mean.shape)
print("Temporal generalization scores shape:", temporal_gen_scores.shape)
print("Cross-decoding scores shape:", cross_decoding_scores.shape)
print("Pre-stimulus prediction scores shape:", pre_stim_scores.shape)
